## 0. Объектно-ориентированное программирование в Python.

Бывает, что вам нужно написать программу, которую вы запустите один раз и которая просто должна решить некоторую задачу. В этом случае все средства хороши, и можно оформлять код так, как вам удобно. Есть и другая ситуация: вам надо запрограммировать что-то, что потом вы будете использовать много раз, а ещё этот код могут захотеть редактировать другие люди. Тут надо задуматься о правильном оформлении кода.

В машинном обучении главное — это модели, их обучение, применение, подсчёт качества и прочие связанные операции. Допустим, вы собрались запрограммировать обучение и применение метода k ближайших соседей и выложить в открытый доступ. Тогда будет логично потребовать от вашего кода следующее:
1. Вы можете что-то улучшить в нём, не поменяв логику работы, и это не сломает ничего у пользователей. Будет странно, если вы, скажем, замените циклы на векторные операции в numpy, опубликуете новую версию, и любой код, зависящий от вашего, перестанет работать. Ведь суть того, что делает ваш код, не поменялась!
2. Достаточно легко можно сделать расширенные версии kNN на основе вашего кода — например, запрограммировать kNN с весами.
3. От пользователей скрыты все детали вашей реализации — чтобы пользоваться вашим кодом для kNN, им не нужно вникать, как вы храните данные, как ищете ближайших соседей и т.д. Они просто вызывают нужные функции, и всё работает.

Объектно-ориентированное программирование (ООП) — это подход к организации кода, который (наверное) лучше всего подходит для оформления операций из машинного обучения. В его основе лежат классы и объекты, а также три важных свойства: инкапсуляция, наследование и полиморфизм. Ниже мы разберёмся со всем этим.

Программист может создать класс, своеобразный чертёж, который будет определять поведение объектов этого класса: какие данные они могут хранить и какой код исполнять. После этого можно создавать объекты класса с ожидаемым поведением.

Поломать голову над формальными определениями ООП можно [на вики](https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BD%D0%BE-%D0%BE%D1%80%D0%B8%D0%B5%D0%BD%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D0%BE%D0%B5_%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5).

Оглавление:
* [Как создать класс](#Как-создать-класс)
* [Методы](#Методы)
* [Атрибуты класса и метод `__init__`](#Атрибуты-класса-и-метод-__init__)
* [Magic методы](#Magic-методы)
* [Копирование](#Копирование)
* [getter, setter](#getter,-setter)
* [`@staticmethod`](#@staticmethod)
* [`@classmethod`](#@classmethod)
* [Наследование, `super()`](#Наследование,-super())
* [ABC — Abstract Base Classes](#ABC-—-Abstract-Base-Classes)
* [Datacasses](#Dataclasses)

<a name="introduction"></a>

In [2]:
class Point2D:
  pass

In [3]:
class Point2D:
    """
    A point in 2D space
    :param x: x coordinate
    :param y: y coordinate
    """
    def __init__(self, x: int, y: int) -> None:
        # Создаём атрибуты и присваиваем им переданные значения координат
        self.x = x
        self.y = y

    def __repr__(self) -> str:
        return f"Point2D({self.x}, {self.y})"

    def __str__(self) -> str:
        return f"A 2D point with coordinates ({self.x}, {self.y})"

    def __add__(self, other: Point2D) -> Point2D:
        return Point2D(self.x + other.x, self.y + other.y)

    def __sub__(self, other: Point2D) -> Point2D:
        return Point2D(self.x - other.x, self.y - other.y)

    def __iadd__(self, other: Point2D) -> Point2D:
        self.x += other.x
        self.y += other.y
        return self

    def __isub__(self, other: Point2D) -> Point2D:
        self.x -= other.x
        self.y -= other.y
        return self

    def __eq__(self, other: Point2D) -> Point2D:
        return self.x == other.x and self.y == other.y

### Копирование

Переменные в питоне всегда хранят ссылку на объект.  Если мы присвоим объект нашего класса двум разным переменным, они обе будут хранить один и тот же объект и "меняться" вместе:

In [4]:
a = Point2D(3, 4)
print(a)

b = a
print(b is a)
b.x = -1

print(a)

print(id(a), id(b))

A 2D point with coordinates (3, 4)
True
A 2D point with coordinates (-1, 4)
137607513551632 137607513551632


Мы поменяли значение в переменной `b`, но значение в `a` тоже поменялось, потому что они ссылаются на один и тот же объект.

Чтобы избежать этого недоразумения в питоне существует модуль `copy` и соответствующая функция:

In [5]:
from copy import copy

a = Point2D(3, 4)
print(a)

b = copy(a)

print(a is b)

b.x = -1

print(a)
print(b)

print(id(a), id(b))

A 2D point with coordinates (3, 4)
False
A 2D point with coordinates (3, 4)
A 2D point with coordinates (-1, 4)
137607513554752 137607513550288


In [9]:
c = [1, [5, 10, 20], 3]
d = copy(c)
d[1][1] = -1
d[1] = 11
c

[1, [5, -1, 20], 3]

In [10]:
from copy import deepcopy
c = [1, [5, 10, 20], 3]
d = deepcopy(c)
d[1][1] = -1
c

[1, [5, 10, 20], 3]

Однако и это не спасёт нас, если в объекте нашего класса снова хранятся изменяемые переменные (например, списки). В таком случае нам понадобится функция `deepcopy`, которая полностью скопирует как сам объект класса, так и всё внутри него, по сути действуя рекурсивно.

Продемонстрируем это на новом классе `Student`:

In [11]:
from typing import Iterable
import copy

class Student:
    """
    A class representing student along with his name and classes he/she takes
    :param name: name of the student
    :param classes: iterable of strings with names of classes
    """
    def __init__(self, name: str, classes: Iterable[str]) -> None:
        self.name = name
        self.classes = classes

    def __repr__(self) -> str:
        return f"Student({repr(self.name)}, {repr(self.classes)})"

student = Student("Ваня", ["Линал", "Алгосы", "Машинное обучение"])
student_deepcopy = copy.deepcopy(student)
student_copy = copy.copy(student)
student_naive = student

# Распечатаем студента
print("До изменений")
print(student)

student_deepcopy.name = "Катя"
student_deepcopy.classes[0] = "Матан"
print("\nИзменения в deepcopy")
print(f"{student_deepcopy}")
print(f"{student}")

student_copy.name = "Лиза"
student_copy.classes[0] = "Экономика"
print("\nИзменения в copy")
print(f"{student_copy}")
print(f"{student}")

student_naive.name = "Лёша"
student_naive.classes[0] = "Алгебра"
print("\nИзменения в простом присваивании")
print(f"{student_naive}")
print(f"{student}")

До изменений
Student('Ваня', ['Линал', 'Алгосы', 'Машинное обучение'])

Изменения в deepcopy
Student('Катя', ['Матан', 'Алгосы', 'Машинное обучение'])
Student('Ваня', ['Линал', 'Алгосы', 'Машинное обучение'])

Изменения в copy
Student('Лиза', ['Экономика', 'Алгосы', 'Машинное обучение'])
Student('Ваня', ['Экономика', 'Алгосы', 'Машинное обучение'])

Изменения в простом присваивании
Student('Лёша', ['Алгебра', 'Алгосы', 'Машинное обучение'])
Student('Лёша', ['Алгебра', 'Алгосы', 'Машинное обучение'])


Как видим, если мы используем просто `copy`, имя студента действительно не меняется в изначальной переменной `student`, потому что соответствующий атрибут `name` был скопирован. А вот список дисциплин меняется, потому что `copy` скопировал в новый объект **ссылку** на этот список, и мы обращаемся к ней из обоих студентов. `deepcopy` позволяет избежать этой проблемы, потому что он создаст новый список.

### getter, setter

Иногда нам хочется, чтобы чтение или установка атрибута объекта сопровождались какой-нибудь логикой, будь то проверка или ещё что. Предположим, мы создаём класс градусника:

In [12]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius
    :param temperature: temperature to contain
    """
    def __init__(self, temperature: float) -> None:
        self.temperature = temperature

У такого класса есть проблема: мы можем присовить ему температуру ниже возможного минимума:

In [13]:
thermometer = Thermometer(10.)
thermometer.temperature = -100000.

In [14]:
del thermometer.temperature
thermometer.temperature

AttributeError: ignored

Мы могли бы решить эту проблему, добавив специальную функцию set_temperature, и сказав, что установка температуры должна происходить только через неё:

In [15]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius
    :param temperature: temperature to contain
    """
    MINIMAL_TEMPERATURE = -273.15
    def __init__(self, temperature: float) -> None:
        self.set_temperature(temperature)

    def set_temperature(self, value: float) -> None:
        if value < Thermometer.MINIMAL_TEMPERATURE:
            raise ValueError(f"Temperature cannot be less than {Thermometer.MINIMAL_TEMPERATURE}")
        self.temperature = value

thermometer = Thermometer(10.)
thermometer.set_temperature(-100000.)

ValueError: ignored

С таким решением, во-первых, мы теряем удобное присвивание по имени атрибута, а во-вторых, весь код, который до этого использовал temperature напрямую теперь будет работать неправильно

Чтобы избежать этих проблем в python есть декоратор `@property`: он позволяет превратить функцию в атрибут и определить поведение при установке и чтения атрибута:

In [22]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius
    :param temperature: temperature to contain
    """
    MINIMAL_TEMPERATURE = -273.15
    def __init__(self, temperature: float) -> None:
        self.temperature = temperature

    def __setattr__(self, name, value) -> None:
        print(name)
        if value < Thermometer.MINIMAL_TEMPERATURE:
            raise ValueError(f"Temperature cannot be less than {Thermometer.MINIMAL_TEMPERATURE}")
        self.__dict__[name] = value

    # def __getattribute__(self, name) -> float:
    #     return self.__dict__[name] * 2

    def __delattr__(self, name):
        pass

thermometer = Thermometer(10.)
thermometer.temperature = 20
print(thermometer.temperature)
del thermometer.temperature
print(thermometer.temperature)

temperature
temperature
20
20


In [24]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius
    :param temperature: temperature to contain
    """
    MINIMAL_TEMPERATURE = -273.15
    def __init__(self, temperature: float) -> None:
        self.temperature = temperature

    @property
    def temperature(self):
        pass

    @temperature.setter
    def temperature(self, value: float) -> None:
        if value < Thermometer.MINIMAL_TEMPERATURE:
            raise ValueError(f"Temperature cannot be less than {Thermometer.MINIMAL_TEMPERATURE}")
        self._temperature = value

    @temperature.deleter
    def temperature(self) -> None:
        pass

    @temperature.getter
    def temperature(self) -> float:
        return self._temperature * 2

thermometer = Thermometer(10.)
thermometer.temperature = 20
print(thermometer.temperature)
del thermometer.temperature
print(thermometer.temperature)

40
40


Разберёмся, как это работает. Во-первых, теперь мы будем хранить нашу реальную температуру в переменной `self._temperature`.
Обращение к этой переменной не будет влечь за собой никакой другой логики, но присваивать ей температуру мы будем только внутри класса, а пользователь снаружи не будет о ней ничего знать и не должен к ней обращаться.

**NB:** Подчёркивание в начале названия атрибута или метода &mdash; это стандартное в питоне обозначение части внутреннего интерфейса класса. Разработчик не обещает, что этот интерфейс не будет меняться в будущих версиях, и пользователь не должен его использовать. Причём сам питон никак не запрещает обращаться к таким атрибутам и методам, просто программисты соблюдают некоторое джентльменское соглашение (цитата из документации)

Сначала мы создаём функцию с названием нашего атрибута (в нашем случае `def temperature`) с декоратором `@property` и в ней возвращаем наше реальное значение температуры (`return self._temperature`). Теперь, когда мы будем обращаться к `thermometer.temperature` будет вызываться эта функция.

Затем мы создаём ещё одну функцию с тем же названием, но теперь с декоратором `@temperature.setter`, в которой проверяем наше условие и присваиваем новое значение нашей внутренней переменной `self._temperature`. Эта функция будет вызываться, когда мы будем присваивать значение `thermometer.temperature`.

### Дескрипторы

Дескрипторы позволяют обобщить это поведение. Допустим, у нас сразу 2 элемента, на которые мы хотим навесить ограничение, чтобы они не были, допустим, отрицательными. Можно прописать свойства для каждого, а можно сделать лучше!

In [25]:
class Order:
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity
    def total(self):
        return self.price * self.quantity

In [32]:
class Order:
    price = NonNegative('price')
    quantity = NonNegative('quantity')
    def __init__(self, name, price, quantity):
        self._name = name
        self.price = price
        self.quantity = quantity
    def total(self):
        return self.price * self.quantity

In [30]:
class NonNegative:
    def __init__(self, name):
        self.name = name
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        instance.__dict__[self.name] = value

In [33]:
order = Order('apple', 10, 5)
print(order.total())
order.quantity = -1
print(order.total())

50


ValueError: ignored

А теперь уберём ещё немного дублирования! (3.6+)

In [35]:
class Order:
    price = NonNegative()
    quantity = NonNegative()
    def __init__(self, name, price, quantity):
        self._name = name
        self.price = price
        self.quantity = quantity
    def total(self):
        return self.price * self.quantity

In [34]:
class NonNegative:
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        instance.__dict__[self.name] = value
    def __set_name__(self, owner, name):
        self.name = name

In [37]:
apple_order = Order('apple', 1, 10)
print(apple_order.total())
apple_order.price = -10

10


ValueError: ignored

### `@staticmethod`

Иногда метод объекта никак не использует его атрибуты (не обращается к self), а просто выполняет независимую логику. В таком случае такой метод называется статическим. Мы можем использовать на нём специальный декоратор `@staticmethod`, и тогда мы помимо прочего сможем вызывать его, даже не создавая объект

Попробуем это на примере нашего термометра. Предположим, мы хотим научить его вычислять выдавать температуру в градусах по Фаренгейту. В таком случае нам следует определить метод `get_fahrenheit` и хорошо бы вынести сам перевод в отдельный метод `celsius_to_fahrenheit`:

In [ ]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius
    :param temperature: temperature to contain
    """
    MINIMAL_TEMPERATURE = -273.15
    def __init__(self, temperature: float) -> None:
        self.temperature = temperature

    @property
    def temperature(self) -> float:
        return self._temperature

    @temperature.setter
    def temperature(self, value: float) -> None:
        if value < Thermometer.MINIMAL_TEMPERATURE:
            raise ValueError(f"Temperature cannot be less than {Thermometer.MINIMAL_TEMPERATURE}")
        self._temperature = value

    def celsius_to_fahrenheit(self, value) -> float:
        return value * 1.8 + 32

    def get_fahrenheit(self) -> float:
        return self.celsius_to_fahrenheit(self.temperature)

thermometer = Thermometer(10.)
print(f"{thermometer.temperature} градусов по Цельсию равны {thermometer.get_fahrenheit()} градусов по Фаренгейту")

10.0 градусов по Цельсию равны 50.0 градусов по Фаренгейту


Наша функция `celsius_to_fahrenheit` просто переводит температуру, которую получает на вход. Она никак не использует `self`. Давайте сделаем её статическим методом.

In [ ]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius
    :param temperature: temperature to contain
    """
    MINIMAL_TEMPERATURE = -273.15
    def __init__(self, temperature: float) -> None:
        self.temperature = temperature

    @property
    def temperature(self) -> float:
        return self._temperature

    @temperature.setter
    def temperature(self, value: float) -> None:
        if value < Thermometer.MINIMAL_TEMPERATURE:
            raise ValueError(f"Temperature cannot be less than {Thermometer.MINIMAL_TEMPERATURE}")
        self._temperature = value

    @staticmethod
    def celsius_to_fahrenheit(value) -> float:
        return value * 1.8 + 32

    def get_fahrenheit(self) -> float:
        return self.celsius_to_fahrenheit(self.temperature)

print(Thermometer.celsius_to_fahrenheit(10))
thermometer = Thermometer(10)
print(f"{thermometer.temperature} градусов по Цельсию равны {thermometer.get_fahrenheit()} градусов по Фаренгейту")

50.0
10 градусов по Цельсию равны 50.0 градусов по Фаренгейту


Обратите внимание, что мы всё так же можем вызывать функцию от объекта класса (как, например, от `self` в `get_fahrenheit`). Но так же теперь мы можем вызывать её и от самого класса:

In [ ]:
Thermometer.celsius_to_fahrenheit(10.)

50.0

### `@classmethod`

Бывает и другая ситуация. Когда метод не использует атрибуты объекта (не обращается к `self`), но использует знание о том, из какого класса он вызывается. Такой метод называется методом класса и мы можем обозначить его декоратором `@classmethod`. Метод с декоратором `@classmethod` принимает в качестве первого аргумента не объект класса (`self`), а сам класс (`cls`).

Предположим, мы хотим уметь создавать наш термометр сразу из температуры по Фаренгейту. Но наш `__init__` принимает температуру в градусах Цельсия. Создадим новый метод класса `from_fahrenheit`, который будет переводить полученную температуру из Фаренгейта в Цельсии (при помощи нового статического метода `fahrenheit_to_celsius` и возвращать новый объект класса:

In [ ]:
class Thermometer:
    """
    Thermometer containing temperature in Celsius
    :param temperature: temperature to contain
    """
    MINIMAL_TEMPERATURE = -273.15
    def __init__(self, temperature: float) -> None:
        self.temperature = temperature

    @property
    def temperature(self) -> float:
        return self._temperature

    @temperature.setter
    def temperature(self, value: float) -> None:
        if value < Thermometer.MINIMAL_TEMPERATURE:
            raise ValueError(f"Temperature cannot be less than {Thermometer.MINIMAL_TEMPERATURE}")
        self._temperature = value

    @staticmethod
    def celsius_to_fahrenheit(value) -> float:
        return value * 1.8 + 32

    @staticmethod
    def fahrenheit_to_celsius(value) -> float:
        return (value - 32) / 1.8

    def get_fahrenheit(self) -> float:
        print(type(self))
        return self.celsius_to_fahrenheit(self.temperature)

    @classmethod
    def from_fahrenheit(cls, temperature_fahrenheit) -> Thermometer:
        temperature_celsius = cls.fahrenheit_to_celsius(temperature_fahrenheit)
        print(type(cls))
        return cls(temperature_celsius)

Теперь мы можем создать термометер из градусов по Фаренгейту:

In [ ]:
thermometer_celsius = Thermometer.from_fahrenheit(100)
print(thermometer_celsius.temperature)
print("%.1000f" % thermometer_celsius.get_fahrenheit())
thermometer_celsius.get_fahrenheit() == 100

<class 'type'>
37.77777777777778
<class '__main__.Thermometer'>
100.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

True

### Наследование, `super()`

Одним из главных механизмов ООП является наследование. Класс может отнаследоваться от другого класса, тем самым получая доступ к его методам и атрибутам. Класс-ребёнок может добавлять свои новые методы и переопределять методы класса-родителя, но так же может использовать и все методы, определённые в классе родителе.

Родительский класс в питоне указывается в скобках после названия класса во время объявления: `class Children(Parent)`. Причём один класс может быть ребёнком сразу нескольких классов. В таком случае они перечисляются через запятую.

Попробуем создать общий класс `Person`, который будет иметь атрибут `name` и сможет представляться при помощи метода `introduce`:

In [40]:
int.__mro__

(int, object)

In [41]:
class Person:
    """
    A class representing a person with a name that can introduce himself
    :param name: the name of the person
    """
    def __init__(self, name: str) -> None:
        self.name = name

    def introduce(self) -> None:
        print(f"Hello, my name is {self.name}")

    def __str__(self) -> str:
        return f"A person named {self.name}"

person = Person("Dima")
person.introduce()
print(person)

Hello, my name is Dima
A person named Dima


In [42]:
type(Person)

type

In [43]:
type(person)

__main__.Person

Теперь создадим подклассы Teacher и Student. Помимо имени у преподавателя будет поле `discipline`, а у студента поле `marks`:

In [44]:
from typing import Dict

class Teacher(Person):
    """
    A class representing a teacher with his discipline
    :param name: the name of the teacher
    :param discipline: the discipline taught by the teacher
    """
    def __init__(self, name: str, discipline: str) -> None:
        super().__init__(name)  # Вызываем __init__ класса-родителя
        self.discipline = discipline

    def __str__(self) -> str:
        return f"A teacher named {self.name} who teaches {self.discipline}"


class Student(Person):
    """
    A class representing a student and his marks
    :param name: the name of the student
    :param marks: a dict of disciplines as keys and marks as values
    """
    def __init__(self, name: str, marks: Dict[str, int]) -> None:
        super().__init__(name)  # Вызываем __init__ класса-родителя
        self.marks = marks

    def __str__(self) -> str:
        return f"A student named {self.name} with marks {self.marks}"

teacher = Teacher("Evgeny", "Machine Learning")
teacher.introduce()
print(teacher)
print()

student = Student("Liza", {"Calculus": 5, "Machine Learning": 10})
student.introduce()
print(student)

Hello, my name is Evgeny
A teacher named Evgeny who teaches Machine Learning

Hello, my name is Liza
A student named Liza with marks {'Calculus': 5, 'Machine Learning': 10}


In [47]:
persons = []
persons.append(person)
persons.append(teacher)
persons.append(student)

for i in persons:
  i.introduce()

Hello, my name is Dima
Hello, my name is Evgeny
I teach Machine Learning
Hello, my name is Liza


Теперь любой код, который использует метод `introduce` может работать как с классом `Person`, так и с классами `Teacher` и `Student`.

И в то же время, ни в одном из этих классов мы не прописывали функцию `introduce`, потому что она уже есть в родительском классе `Person`, так что мы избежали дублирования кода и объединили объекты со схожей логикой между собой.

Стоит обратить особое внимание на оператор `super()`. Все методы, вызывающиеся от `super()`, будут вызваны от родительского класса (классов) объекта. Таким образом, когда мы вызываем `super().__init__(name)` в классах `Teacher` и `Student` мы выполняем код из метода `__init__` родительского класса `Person`: `self.name = name`, то есть определяем в объекте атрибут `name`. А затем уже мы добавляем новые атрибуты. Если функция `__init__` в родительском классе как-то поменяется, у нас ничего не сломается, и эти изменения будут использоваться и в дочерних классах.

Так же следует обратить внимание на то, что метод `__str__` определён как в родительсом классе `Person`, так и в дочерних `Teacher` и `Student`. Таким образом, дочерние классы переопределяют этот метод, и при вызове `print` или `str` от объектов дочерних классов будет вызыван их метод `__str__`, а не родительский.

Мы могли бы использовать `super()` и не только в `__init__`, например, чтобы расширить функционал представления:

In [52]:
class Teacher(Person):
    """
    A class representing a teacher with his discipline
    :param name: the name of the teacher
    :param discipline: the discipline taught by the teacher
    """
    def __init__(self, name: str, discipline: str) -> None:
        super().__init__(name)  # Вызываем __init__ класса-родителя
        self.discipline = discipline

    def introduce(self) -> None:
        super().introduce()
        print(f"I teach {self.discipline}")

    def __str__(self) -> str:
        return f"A teacher named {self.name} who teaches {self.discipline}"

teacher = Teacher("Evgeny", "Machine Learning")
teacher.introduce()

Hello, my name is 10
I teach Machine Learning


Кроме того мы могли бы определить в дочерних классах совершенно новые методы, которых нет в классе-родителе:

In [57]:
class Student(Person, Thermometer):
    """
    A class representing a student and his marks
    :param name: the name of the student
    :param marks: a dict of disciplines as keys and marks as values
    """
    def __init__(self, name: str, marks) -> None:
        Person.__init__(self, name)  # Вызываем __init__ класса-родителя
        Thermometer.__init__(self, 36.6)
        self.marks = marks

    def __str__(self) -> str:
        return f"A student named {self.name} with marks {self.marks}"

    def get_fails(self):
        """
        Gets failed disciplines
        """
        return {discipline: mark for discipline, mark in self.marks.items() if mark < 4}

student = Student("Liza", {"Calculus": 3, "Machine Learning": 10})
student.introduce()
print(f"Failed disciplines: {student.get_fails()}")
student.temperature

Hello, my name is Liza
Failed disciplines: {'Calculus': 3}


73.2

Мы разобрали очень простой пример наследования. На практике деревья наследования могут быть очень большими, один класс может иметь множество детей и множество родителей. В таких случаях может стать непонятно, в каком именно порядке питон обходит родителей класса, чтобы понять, чью реализацию того или иного метода использовать.

Чтобы посмотреть этот порядок можно использовать атрибут **класса** `__mro__` (Method Resolution Order):

In [49]:
Student.__mro__

(__main__.Student, __main__.Person, object)

Мы видим, что если мы попытаемся вызвать у объекта класса `Student`, например, метод `introduce`, то сначала питон попытается найти его реализацию в самом классе `Student`. Если там её не окажется, он попробует класс `Person`, а если и там нет, то родительский класс всех классов `object`. (А если и там нет, то выдаст `AttributeError`)

Подробнее про то, как именно питон определяет этот порядок, можно почитать, например, [тут](https://habr.com/ru/post/62203/).

### ABC &mdash; Abstract Base Classes

Бывает так, что мы хотим обозначить какой-то список методов, которые должны реализовывать все классы одного семейства, но не можем предоставить дефолтной реализации для этих методов. Например, нам нужно написать код, который вычислял бы сумму площадей или периметров списка фигур, но для каждого типа фигуры эти величины вычисляются по-своему, а всё же хотелось бы как-то объединить их всех в один родительский класс и показать, какими методами должна обладать фигура.

В таком случае нам на помощь приходит модуль `abc` и декоратор `@abstractmethod`. Если мы отнаследуем наш класс от класса `abc.ABC` и добавим к методам декоратор `@abc.abstractmethod`, все классы, наследующиеся от нашего будут обязаны объявить эти методы, а иначе питон выдаст ошибку.

Давайте поробуем:

In [58]:
import abc

class Shape(abc.ABC):
    """
    Shape class capable of calculating its area and perimeter
    """
    @abc.abstractmethod
    def get_area(self) -> float:
        """
        Method for getting the area of the shape
        """
        pass

    @abc.abstractmethod
    def get_perimeter(self) -> float:
        """
        Method for getting the perimeter of the shape
        """
        pass

Попробуем создать класс, который наследуется от `Shape`, но не реализует эти методы:

In [59]:
bad_shape = Shape()

TypeError: ignored

In [60]:
class BadShape(Shape):
    pass

bad_shape = BadShape()

TypeError: ignored

Как видим питон не дал нам создать объект такого класса (но сам класс создать дал!)

Так же мы не сможем создать объект самого абстрактного класса `Shape`:

In [ ]:
shape = Shape()

TypeError: ignored

Теперь реализуем нормальные фигуры:

In [61]:
import math

class Circle(Shape):
    """
    A circle with a some radius
    """
    def __init__(self, radius: float) -> None:
        self.radius = radius

    @property
    def radius(self) -> float:
        return self._radius

    @radius.setter
    def radius(self, value: float) -> None:
        if value < 0:
            raise ValueError(f"Radius has to be >= 0")
        self._radius = value

    @radius.deleter
    def radius(self) -> None:
        pass

    def get_area(self) -> float:
        return math.pi * self.radius ** 2

    def get_perimeter(self) -> float:
        return 2 * math.pi * self.radius


class Rectangle(Shape):
    """
    A rectangle with some width and height
    """
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height

    def get_area(self) -> float:
        return self.width * self.height

    def get_perimeter(self) -> float:
        return 2 * (self.width + self.height)

Теперь мы можем написать функции, которая будет работать с нашими объектами `Shape`, и они могут быть уверены, что все классы, наследующиеся от `Shape` будут реализовывать нужные им методы:

In [62]:
def get_areas_sum(shapes):
    # Проверяем, что все наши объекты наследуются от Shape:
    for shape in shapes:
        if not isinstance(shape, Shape):
          raise ValueError(f"Only Shape objects are allowed, you tried to pass object of type {type(shape)}")
    # if not all(isinstance(shape, Shape) for shape in shapes):
    #   raise ValueError("Only Shape objects are allowed")
    return sum(shape.get_area() for shape in shapes)

def get_perimeters_sum(shapes):
    # Проверяем, что все наши объекты наследуются от Shape:
    if not all(isinstance(shape, Shape) for shape in shapes):
        raise ValueError("Only Shape objects are allowed")
    return sum(shape.get_perimeter() for shape in shapes)

Попробуем использовать наши функции по назначению:

In [63]:
my_circle = Circle(10)
del my_circle.radius

In [64]:
print(get_areas_sum([Circle(1.), Rectangle(1., 1.)]))
print(get_areas_sum([Circle(1.), Circle(2.), Rectangle(3., 2.), my_circle]))

print(get_perimeters_sum([Circle(1.), Rectangle(1., 1.)]))
print(get_perimeters_sum([Circle(1.), Circle(2.), Rectangle(3., 2.)]))

4.141592653589793
335.8672286269283
10.283185307179586
28.84955592153876


Попробуем подать неправильные объекты:

In [ ]:
print(get_areas_sum([Circle(1.), Rectangle(1., 1.), "STRING"]))

ValueError: ignored

Питон так же имеет множество встроенных абстрактных классов, которые вы можете использовать, для проверки того, что объект удовлетворяет определённому интерфейсу: https://docs.python.org/3/library/collections.abc.html

In [ ]:
from collections.abc import *
class MyStorage(Sequence):
  def __getitem__(self, instance):
    pass
  def __len__(self):
    pass

storage = MyStorage()
# dir(storage)

# for i in storage:
#   print(i)